# Introduction

This notebook gives a brief intro to using holoviz-based plotting, incl our own tools that work with it.

To make sure interactivity works correctly, run this notebook in an environment that has all the right stuff installed (i.e., your labcore environment)

## Some basic preliminaries

We assume that `raw` data is in DataDict formats.
For processing and plotting we convert to either `pandas.DataFrame` (for ungridded data) or `xarray.Dataset` (for gridded data).
This approach should be the general way we handle data in the lab.

# Brief primer on vanilla holoviz

We have some tools on top of it for general easy data exploration. 

But for data analysis it's probably very wise for everyone to learn how to use holoviz directly. 
It allows making custom plots extremely easily. 

Docs are here: https://holoviz.org; see in particular ``holoviews`` and `hvplot`.

Here is a very simple example -- we use synthetic Rabi/Chevron data for that.
But much more is possible -- check out the examples in the documentation for how to do different things.

In [1]:
# first, make a dataset in datadict
import numpy as np

from labcore.testing.dispersive_qubit_readout_data import chevron_dataset

raw_data = chevron_dataset(
    Omega_0=1e6,
    Delta_vals=np.linspace(-1e6, 1e6, 11),
    t_vals=np.linspace(0, 3e-6, 25),
    n=500,  # number of shots
)

raw_data

signal: (275, 500)
  ⌙ repetition: (275, 500)
  ⌙ detuning (Hz): (275,)
  ⌙ time (s): (275,)

In [2]:
# next -- convert into xarray. since we know its a nice regular grid, that's the easiest choice here.
from labcore.data.datadict import datadict_to_meshgrid, dd2xr
from labcore.analysis import split_complex

# with the holoviz (or all pydata) tools, complex data isn't always the best, so we split it immediately.
xrdata = split_complex(
    dd2xr(datadict_to_meshgrid(raw_data))
)
xrdata

<xarray.Dataset>
Dimensions:     (repetition: 500, detuning: 11, time: 25)
Coordinates:
  * repetition  (repetition) int64 1 2 3 4 5 6 7 ... 494 495 496 497 498 499 500
  * detuning    (detuning) float64 -1e+06 -8e+05 -6e+05 ... 6e+05 8e+05 1e+06
  * time        (time) float64 0.0 1.25e-07 2.5e-07 ... 2.75e-06 2.875e-06 3e-06
Data variables:
    signal_Re   (repetition, detuning, time) float64 1.963 1.333 ... 2.531 1.721
    signal_Im   (repetition, detuning, time) float64 -0.1405 -0.5443 ... -0.1905

In [4]:
# now we can directly use hvplot on this
import hvplot.xarray  # this import patches xarray objects so we can use hvplot on them.

from bokeh.models import CrosshairTool

# plot: average over repetition, then plot the real part as 2d map.
xrdata.signal_Re.mean('repetition').hvplot.quadmesh(
    x='detuning',
    y='time',
)

:QuadMesh   [detuning,time]   (signal_Re)

In [4]:
# it's also pretty easy to make linecuts with sliders
_data = xrdata.mean('repetition')

# here: overlay line and scatter plot
# can change x to get a different cut
_data.hvplot.line(x='time') * _data.hvplot.scatter(x='time')

:DynamicMap   [detuning]
   :Overlay
      .NdOverlay.I  :NdOverlay   [Variable]
         :Curve   [time]   (value)
      .NdOverlay.II :NdOverlay   [Variable]
         :Scatter   [time]   (value)

In [5]:
# looking at readout histograms -- use simply all points in the set
# hexbin is a very simple way to have a quick look at this. 
# doesn't offer too much control, but is 
xrdata.hvplot(
    kind='hexbin',
    aspect=1,
    groupby=[],
    x='signal_Re',
    y='signal_Im',
)

:HexTiles   [signal_Re,signal_Im]

In [6]:
# if we want to select based on some time/detuning values...
xrdata.hvplot(
    kind='hexbin',
    aspect=1,
    groupby=['time', 'detuning'],
    x='signal_Re',
    y='signal_Im',
    xlim=(-3,3),
    ylim=(-3,3)
)

:DynamicMap   [time,detuning]
   :HexTiles   [signal_Re,signal_Im]

# Using our own data exploration tools

## The tldr;

A very simple tool for data exploration exists -- it works almost like plottr, but inside jupyter notebooks.

In [7]:
# a very simple app for exploring typical data
# it has a few issues still -- sometimes it requires going back to setting x and y to None to reset sliders...


from labcore.analysis.plotting.holo import plot_data

plot_data(xrdata.mean('repetition'))

Column
    [0] Column()
    [1] ParamMethod(method, _pane=Column, defer_load=False)

## loading data and apps

In [7]:
import panel as pn
from labcore.data.datadict import DataDict
from labcore.analysis.plotting.holo import Node, LoaderNodeBase
from labcore.testing.dispersive_qubit_readout_data import chevron_dataset

class RabiSimulation(LoaderNodeBase):
    def load_data(self) -> DataDict:
        Omega_0 = 1
        Delta_vals = np.linspace(-2, 2, 41)
        t_vals = np.linspace(0, 3, 31)
        nreps = 50
        dd = chevron_dataset(Omega_0, Delta_vals, t_vals, nreps).expand()
        return dd       

s = RabiSimulation(name='sim')
app = pn.Column(
    s,
    s.plot,
)
app

Column
    [0] Column
        [0] Row
            [0] Column
                [0] StaticText(margin=(5, 10, 0, 10), value='Pre-processing')
                [1] RadioButtonGroup(margin=(0, 10, 5, 10), name='Pre-processing', options=[None, 'Average'], value='Average')
            [1] TextInput(name='Pre-process dimension', value='repetition')
        [1] Checkbox(name='Auto-grid', value=True)
        [2] Button(name='Load data')
    [1] ParamMethod(method, _pane=Column, defer_load=False)

In [2]:
import panel as pn
from labcore.analysis.plotting.holo import LoaderNodePath

s = LoaderNodePath(name='load data' , path="./data/2023-11-16T073053_5d6e89ca-qA_power_rabi/data.ddh5")
app = pn.Column(
    s,
    s.plot,
)
app

Column
    [0] Column
        [0] Row
            [0] Column
                [0] StaticText(margin=(0, 10, 0, 10), value='Pre-processing')
                [1] RadioButtonGroup(margin=(0, 10, 0, 10), name='Pre-processing', options=[None, 'Average'], value='Average')
            [1] TextInput(name='Pre-process dimension', value='repetition')
        [1] Row
            [0] FloatSlider(end=10, name='Refresh Rate (Seconds)', start=1, step=1, value=1)
            [1] Toggle(name='Pause Refresh')
        [2] TextInput(name='File Location', value='./data/2023-11-16T073053_...)
        [3] Checkbox(name='Auto-grid', value=True)
        [4] Button(name='Load data')
    [1] ParamMethod(method, _pane=Column, defer_load=False)